In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [2]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [36]:
#LIST_PATH
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

63


In [37]:
dataset[0]

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 11 a 30/2017014 CR 014/103468016 Scintigraphie sur TEP TDM/CT CT TAP 2.0 I30f 3',
 'CT',
 '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 11 a 30/2017014 CR 014/103468016 Scintigraphie sur TEP TDM/PT TEP TAP Corrig',
 'PT',
 '1.2.250.1.38.2.1.102.140724093005.103468016',
 '2017014',
 'cr_014_jul 25_2014.csv']

In [ ]:
#BUILD NIFTI FROM DATASET 1 2 et 3

json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/json_dataset_4/dataset_4.json'
dataset_4 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_4.append(info)

print(len(dataset_4))

#REWRITE GOOD PATH

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('salim', 'deeplearning')

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('DD 2To', '78ca2911-9e9f-4f78-b80a-848024b95f92') 

write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET2', 'dataset', dataset)

In [40]:
#remove remove_bi_file

paths = get_series_path('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM')


In [43]:
from library_dicom.dicom_processor.tools.folders import *
for path in paths : 
    remove_bi_file(path)



In [39]:
#On met les erreurs de coté
#generer mip en meme temps 

nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

csv_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/CSV'
mip_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/MIP'



false_mask_dataset_1 = []
error_dataset_1 = []
path_mip = []

results_false_mask = []

for serie in dataset: 
    try : 
        #print(serie)
        print(dataset.index(serie))
        subliste = []

        results = []

        if serie[1] == 'PT' : 
            
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']



            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                    #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")
                #shutil.rmtree(os.path.join(dicom_directory+'/'+serie[-2], serie[]))
                #print("dicom effacé")

            else : 
                print("FALSE MASK")
                false_mask_dataset_1.append(serie)
                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))
                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))
                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))
                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                subliste.append(angle_filename_mask)

                path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                     #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                print("EXPORT NIFTI MASK")
                #shutil.rmtree(os.path.join(dicom_directory, serie[-2]))
                #print('dicom effacé')
            else : 
                print("FALSE MASK")
                false_mask_dataset_1.append(serie)
                print(mask_objet.calcul_suv(nifti_array))
                results.append(mask_objet.calcul_suv(nifti_array))
                print(mask_objet.ecart_suv_max(nifti_array))
                results.append(mask_objet.ecart_suv_max(nifti_array))
                print(mask_objet.ecart_suv_mean(nifti_array))
                results.append(mask_objet.ecart_suv_mean(nifti_array))
                print(mask_objet.ecart_SD(nifti_array))
                results.append(mask_objet.ecart_SD(nifti_array))

                results_false_mask.append(results)
                mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                subliste.append(angle_filename)
                angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                subliste.append(angle_filename_mask)

                path_mip.append(subliste)
               

    except Exception as err : 
        print(err)
        print(serie)
        error_dataset_1.append(serie)


0
MASK CORRECT
in method 'Image_SetSpacing', argument 2 of type 'std::vector< double,std::allocator< double > > const &'
['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017031 CR 031/Unknown Study/PT p8273s0_FUE_ctac_resamp.img 3DRAMLA', 'PT', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017031 CR 031/Unknown Study/CT', 'CT', '1.2.840.113704.1.111.920.1223892920.6', '2017031', 'cr_031_oct 13_2008.csv']
1
MASK CORRECT
EXPORT NIFTI PT
in method 'Image_SetSpacing', argument 2 of type 'std::vector< double,std::allocator< double > > const &'
['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017042 CR 042/A22514015868 TEP corps entier/CT CT TAP 3.0 B30s', 'CT', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Patient 31 à 50/2017042 CR 042/A22514015868 TEP corps entier/PT PET TAP AC HD', 'PT', '1.2.840.113711.41813.3.11624.395627943.26.2116281012.16890', '2

In [45]:
print(len(error_dataset_1))
print(len(false_mask_dataset_1))

10
13


In [46]:
#SAVE THE SERIE WITH FALSE MASK 
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI', 'false_mask', false_mask_dataset_1)
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI', 'false_mask_results', results_false_mask)

write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI', 'error', error_dataset_1)
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/MIP', 'path_mip', path_mip)

In [47]:
#create pdf 
filename = os.path.join(mip_directory, 'list_mip_dataset.pdf')
create_pdf_mip(path_mip, filename)

In [40]:
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/false_mask.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

13


In [42]:
serie = dataset[9]
print(serie)

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Derniers patients/2017097 CR 097/A20080862074 CORPS ENTIER AU FDG/CT CT TAP 55 B30s', 'CT', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Derniers patients/2017097 CR 097/A20080862074 CORPS ENTIER AU FDG/PT PET TAP AC HD', 'PT', '2.16.840.1.113669.632.20.310000.20001712333', '2017097', 'cr_097_dec 22_2009.csv', 'cr_097_dec 7_2011.csv']


In [44]:
serie_pt_objet = SeriesPT(serie[2])
nifti_array = serie_pt_objet.get_numpy_array()
study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

size_matrix = serie_pt_objet.get_size_matrix()
mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)



In [45]:
mask_objet.ecart_suv_max(nifti_array)

[1, 'POLYGON', 0.7599999999999998]

In [46]:
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/false_mask.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

14


In [47]:
#problème de bords pas pris en compte => mip ok donc on genere nifti
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

csv_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/CSV'
mip_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/MIP'

for serie in dataset: 
    try : 
        #print(serie)
        print(dataset.index(serie))

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']



            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            #threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            #if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")
                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
            #shutil.rmtree(os.path.join(dicom_directory, serie[-2]))
            #print("dicom effacé")

            #else : 
                #print("FALSE MASK")
                #false_mask_dataset_4.append(serie)
                #print(mask_objet.calcul_suv(nifti_array))
                #results.append(mask_objet.calcul_suv(nifti_array))
                #print(mask_objet.ecart_suv_max(nifti_array))
                #results.append(mask_objet.ecart_suv_max(nifti_array))
                #print(mask_objet.ecart_suv_mean(nifti_array))
                #results.append(mask_objet.ecart_suv_mean(nifti_array))
                #print(mask_objet.ecart_SD(nifti_array))
                #results.append(mask_objet.ecart_SD(nifti_array))

                #results_false_mask.append(results)
                #mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                #angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                #subliste.append(angle_filename)
                #angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                #subliste.append(angle_filename_mask)

                #path_mip.append(subliste)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array

            number_roi = mask_4D.shape[3]
            #threshold = mask_objet.details_rois['SUVlo']
        #seuillage


            #if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
            print("MASK CORRECT")
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                     #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")
                    #generation nifti mask
            filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
            print("EXPORT NIFTI MASK")
            #shutil.rmtree(os.path.join(dicom_directory, serie[-2]))
            #print('dicom effacé')
            #else : 
                #print("FALSE MASK")
                #false_mask_dataset_4.append(serie)
                #print(mask_objet.calcul_suv(nifti_array))
                #results.append(mask_objet.calcul_suv(nifti_array))
                #print(mask_objet.ecart_suv_max(nifti_array))
                #results.append(mask_objet.ecart_suv_max(nifti_array))
                #print(mask_objet.ecart_suv_mean(nifti_array))
                #results.append(mask_objet.ecart_suv_mean(nifti_array))
                #print(mask_objet.ecart_SD(nifti_array))
                #results.append(mask_objet.ecart_SD(nifti_array))

                #results_false_mask.append(results)
                #mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array)

                #angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
                #subliste.append(angle_filename)
                #angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
                #subliste.append(angle_filename_mask)

                #path_mip.append(subliste)
               

    except Exception as err : 
        print(err)
        print(serie)
        error_dataset_1.append(serie)

0
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
1
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
2
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
3
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
4
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
5
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
6
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
7
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
8
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
9
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
10
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
11
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
12
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
13
MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK


In [9]:
#ON VA TESTER LE FLIP SUR LES FALSE MASK 
#On met les erreurs de coté

error_2 = []
false_mask_after_flip = []

path_mip = []

mip_directory = '/media/salim/DD 2To/GAINED_MIP_FLIP'


for serie in false_mask : 
    try : 
        #print(serie)
        print(false_mask.index(serie))
        subliste = []

        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
            subliste.append(angle_filename)


            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array
            mask_flip = mask_objet.flip_z(mask_4D)


            number_roi = mask_flip.shape[3]
            threshold = mask_objet.details_rois['SUVlo']
        #seuillage
            mask_flip = threshold_mask(mask_flip, mask_objet.details_rois, nifti_array)
            angle_filename_mask = mip_projection_4D(mask_flip, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
            subliste.append(angle_filename_mask)

            path_mip.append(subliste)

            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK FLIPPE CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                    #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_flip)
                print("EXPORT NIFTI MASK")
            else : 
                print("FALSE MASK")
                print(mask_objet.calcul_suv(nifti_array))
                print("ECART SUV MAX")
                print(mask_objet.ecart_suv_max(nifti_array))
                print("ECART SUV MEAN")
                print(mask_objet.ecart_suv_mean(nifti_array))
                print("ECART SD")
                print(mask_objet.ecart_SD(nifti_array))
                false_mask_after_flip.append(serie)


        else : 
            serie_pt_objet = SeriesPT(serie[2])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
            subliste.append(angle_filename)


            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[6]), size_matrix)
            mask_4D = mask_objet.mask_array
            mask_flip = mask_objet.flip_z(mask_4D)

            number_roi = mask_flip.shape[3]
            threshold = mask_objet.details_rois['SUVlo']
        #seuillage
            mask_flip = threshold_mask(mask_flip, mask_objet.details_rois, nifti_array)
            angle_filename_mask = mip_projection_4D(mask_flip, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
            subliste.append(angle_filename_mask)

            path_mip.append(subliste)

            if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                print("MASK FLIPPE CORRECT")
                    #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                     #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                    #generation nifti mask
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_flip)
                print("EXPORT NIFTI MASK")
            else : 
                print("FALSE MASK")
                print(mask_objet.calcul_suv(nifti_array))
                print("ECART SUV MAX")
                print(mask_objet.ecart_suv_max(nifti_array))
                print("ECART SUV MEAN")
                print(mask_objet.ecart_suv_mean(nifti_array))
                print("ECART SD")
                print(mask_objet.ecart_SD(nifti_array))
                false_mask_after_flip.append(serie)
               
                
                   


    except Exception as err : 
        print(err)
        print(serie)
        error_2.append(serie)

        
write_json_file('/media/salim/DD 2To/GAINED_NIFTI/json', 'GAINED_false_mask_after_flip', false_mask_after_flip)
write_json_file('/media/salim/DD 2To/GAINED_NIFTI/json', 'GAINED_error_2', error_2)


write_json_file('/media/salim/DD 2To/GAINED_MIP_FLIP', 'GAINED_list_mip_flip', path_mip)
filename = os.path.join(mip_directory, 'list_mip_flip.pdf')
create_pdf_mip(path_mip, filename)

0
taille mask :  (144, 144, 234, 9)
FALSE MASK
{1: {'SUV_max': 10.28, 'SUV_mean': 6.0, 'SD': 1.72}, 2: {'SUV_max': 37.34, 'SUV_mean': 24.37, 'SD': 4.14}, 3: {'SUV_max': 26.18, 'SUV_mean': 15.4, 'SD': 3.77}, 4: {'SUV_max': 32.71, 'SUV_mean': 22.52, 'SD': 5.02}, 5: {'SUV_max': 30.35, 'SUV_mean': 21.08, 'SD': 4.86}, 6: {'SUV_max': 31.35, 'SUV_mean': 19.74, 'SD': 5.0}, 7: {'SUV_max': 25.81, 'SUV_mean': 15.53, 'SD': 3.53}, 8: {'SUV_max': 16.15, 'SUV_mean': 10.26, 'SD': 2.65}, 9: {'SUV_max': 15.11, 'SUV_mean': 9.41, 'SD': 2.42}}
ECART SUV MAX
[9, 'ELLIPSE', 4.83]
ECART SUV MEAN
[9, 'ELLIPSE', 3.41]
ECART SD
[9, 'ELLIPSE', 0.6699999999999999]
1
taille mask :  (144, 144, 255, 49)
FALSE MASK
{1: {'SUV_max': 16.91, 'SUV_mean': 10.4, 'SD': 2.69}, 2: {'SUV_max': 11.45, 'SUV_mean': 7.16, 'SD': 1.88}, 3: {'SUV_max': 17.59, 'SUV_mean': 11.97, 'SD': 2.92}, 4: {'SUV_max': 8.37, 'SUV_mean': 5.02, 'SD': 1.65}, 5: {'SUV_max': 21.3, 'SUV_mean': 12.17, 'SD': 3.14}, 6: {'SUV_max': 12.15, 'SUV_mean': 7.08, 'S

KeyboardInterrupt: 

In [ ]:
#GENERER MIP DES MASK FAUX
mip_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_MIP/DATASET_1'

path_mip = []
for serie in dataset_coronal : 
    index = dataset_coronal.index(serie)
    print(index)
    subliste = []
    if serie[1] =='PT' :
        serie_pt_objet = SeriesPT(serie[0])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
        subliste.append(angle_filename)

        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        mask_4D = mask_objet.mask_array

        print(mask_4D.shape)
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage
        flip_mask = mask_objet.flip_z(mask_4D)
        print(flip_mask.shape)
        #mask_4D_flip = mask_objet.mask_array
        new_mask_4D_flip = threshold_mask(flip_mask, mask_objet.details_rois, nifti_array)
        angle_filename_mask = mip_projection_4D(new_mask_4D_flip, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)


    else : 
        serie_pt_objet = SeriesPT(serie[2])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0) 
        #mip.png PET
        subliste.append(angle_filename)
        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        
        mask_4D = mask_objet.mask_array

        print(mask_4D.shape)
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage

        flip_mask = mask_objet.flip_z(mask_4D)
        
        print(flip_mask.shape)
        #mask_4D_flip = mask_objet.mask_array
        new_mask_4D_flip = threshold_mask(flip_mask, mask_objet.details_rois, nifti_array)
        angle_filename_mask = mip_projection_4D(new_mask_4D_flip, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)


In [ ]:
#lire json false_mask 

In [ ]:
#GENERER MIP MASK CORONAL 

In [6]:
#ALGO MIP FLIP 

mip_directory = '/media/salim/DD 2To/AHL2011_MIP_FLIP'

path_mip = []
for serie in dataset_coronal : 
    index = dataset_coronal.index(serie)
    print(index)
    subliste = []
    if serie[1] =='PT' :
        serie_pt_objet = SeriesPT(serie[0])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0)             #mip.png PET
        subliste.append(angle_filename)

        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        mask_4D = mask_objet.mask_array

        print(mask_4D.shape)
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage
        flip_mask = mask_objet.flip_z(mask_4D)
        print(flip_mask.shape)
        #mask_4D_flip = mask_objet.mask_array
        new_mask_4D_flip = threshold_mask(flip_mask, mask_objet.details_rois, nifti_array)
        angle_filename_mask = mip_projection_4D(new_mask_4D_flip, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)


    else : 
        serie_pt_objet = SeriesPT(serie[2])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 5.0) 
        #mip.png PET
        subliste.append(angle_filename)
        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        
        mask_4D = mask_objet.mask_array

        print(mask_4D.shape)
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage

        flip_mask = mask_objet.flip_z(mask_4D)
        
        print(flip_mask.shape)
        #mask_4D_flip = mask_objet.mask_array
        new_mask_4D_flip = threshold_mask(flip_mask, mask_objet.details_rois, nifti_array)
        angle_filename_mask = mip_projection_4D(new_mask_4D_flip, 0, mip_directory, study_uid, number_roi, borne_max = 1.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)

0
(144, 144, 234, 21)
(144, 144, 234, 21)
taille mask :  (144, 144, 234, 21)
1
(256, 256, 299, 11)
(256, 256, 299, 11)
taille mask :  (256, 256, 299, 11)
2
(144, 144, 255, 4)
(144, 144, 255, 4)
taille mask :  (144, 144, 255, 4)
3
(256, 256, 335, 25)
(256, 256, 335, 25)
taille mask :  (256, 256, 335, 25)
4
(256, 256, 515, 12)
(256, 256, 515, 12)
taille mask :  (256, 256, 515, 12)
5
(144, 144, 276, 15)
(144, 144, 276, 15)
taille mask :  (144, 144, 276, 15)
6
(144, 144, 276, 15)
(144, 144, 276, 15)
taille mask :  (144, 144, 276, 15)
7
(144, 144, 255, 15)
(144, 144, 255, 15)
taille mask :  (144, 144, 255, 15)
8
(144, 144, 234, 11)
(144, 144, 234, 11)
taille mask :  (144, 144, 234, 11)
9
(168, 168, 405, 32)
(168, 168, 405, 32)
taille mask :  (168, 168, 405, 32)
10
(144, 144, 255, 5)
(144, 144, 255, 5)
taille mask :  (144, 144, 255, 5)


In [15]:
#POUR PET 2 ET PET 4 (SANS MASK)

#RELOAD JSON FROM PARSED DICOM IPYNB


json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET2/GAINED_list_path_CT_PET2.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

548


In [16]:
dataset[500]

['/media/salim/DD 2To/GAINED_Validated_DICOMS/51011101871008/1.2.250.1.38.2.1.102.140623080607.103379464/1.3.12.2.1107.5.99.1.123456.2.0.34151922895497/1.3.12.2.1107.5.99.1.123456.2.0.34151922895497',
 'PT',
 '/media/salim/DD 2To/GAINED_Validated_DICOMS/51011101871008/1.2.250.1.38.2.1.102.140623080607.103379464/1.3.12.2.1107.5.1.4.41145.4.0.177289120842559/1.3.12.2.1107.5.1.4.41145.4.0.177289120842559',
 'CT',
 '1.2.250.1.38.2.1.102.140623080607.103379464',
 '51011101871008']

In [17]:
#REWRITE GOOD PATH

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('salim', 'deeplearning')

for serie in dataset : 
    for i in range(0,3,2) :
        serie[i] = serie[i].replace('DD 2To', 'Elements') 

In [18]:
dataset[500]

['/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101871008/1.2.250.1.38.2.1.102.140623080607.103379464/1.3.12.2.1107.5.99.1.123456.2.0.34151922895497/1.3.12.2.1107.5.99.1.123456.2.0.34151922895497',
 'PT',
 '/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101871008/1.2.250.1.38.2.1.102.140623080607.103379464/1.3.12.2.1107.5.1.4.41145.4.0.177289120842559/1.3.12.2.1107.5.1.4.41145.4.0.177289120842559',
 'CT',
 '1.2.250.1.38.2.1.102.140623080607.103379464',
 '51011101871008']

In [5]:
#CHECKER LES UNCONSTANT SPACING
unconstant_spacing = []
serie_error = []
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] =='PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing PET")
                unconstant_spacing.append(serie)
            else : 
                serie_ct_objet = SeriesCT(serie[2])
                nifti_array = serie_ct_objet.get_numpy_array()
                if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                    unconstant_spacing.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2])
            nifti_array = serie_pt_objet.get_numpy_array()
            if serie_pt_objet.get_z_spacing =='Unconstant Spacing' :
                print("Unconstant Spacing CT")
                unconstant_spacing.append(serie)
            else : 
                serie_ct_objet = SeriesCT(serie[0])
                nifti_array = serie_ct_objet.get_numpy_array()
                if serie_ct_objet.get_z_spacing =='Unconstant Spacing' :
                    unconstant_spacing.append(serie)

    except Exception as err : 
        print(serie)
        print(err)
        serie_error.append(serie)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
#GENERATION NIFTI CT PET DES PET 2/4

nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET2'
for serie in dataset : 
    print(dataset.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[2])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")



        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']

            
                    #generation nifti PT
            filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
            serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
            print("EXPORT NIFTI PT")
                    #generation nifti CT
            serie_ct_objet = SeriesCT(serie[0])
            serie_ct_objet.get_numpy_array()
            filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
            serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
            print("EXPORT NIFTI CT")
 

    except Exception as err : 
        print(serie)
        print(err)

EXPORT NIFTI PT
EXPORT NIFTI CT
685
EXPORT NIFTI PT
EXPORT NIFTI CT
686
EXPORT NIFTI PT
EXPORT NIFTI CT
687
EXPORT NIFTI PT
EXPORT NIFTI CT
688
EXPORT NIFTI PT
EXPORT NIFTI CT
689
EXPORT NIFTI PT
EXPORT NIFTI CT
690
EXPORT NIFTI PT
EXPORT NIFTI CT
691
EXPORT NIFTI PT
EXPORT NIFTI CT
692
EXPORT NIFTI PT
EXPORT NIFTI CT
693
EXPORT NIFTI PT
EXPORT NIFTI CT
694
EXPORT NIFTI PT
EXPORT NIFTI CT
695
EXPORT NIFTI PT
EXPORT NIFTI CT
696
EXPORT NIFTI PT
EXPORT NIFTI CT
697
EXPORT NIFTI PT
EXPORT NIFTI CT
698
EXPORT NIFTI PT
EXPORT NIFTI CT
699
EXPORT NIFTI PT
EXPORT NIFTI CT
700
EXPORT NIFTI PT
EXPORT NIFTI CT
701
EXPORT NIFTI PT
EXPORT NIFTI CT
702
EXPORT NIFTI PT
EXPORT NIFTI CT
703
EXPORT NIFTI PT
EXPORT NIFTI CT
704
EXPORT NIFTI PT
EXPORT NIFTI CT
705
EXPORT NIFTI PT
EXPORT NIFTI CT
706
EXPORT NIFTI PT
EXPORT NIFTI CT
707
EXPORT NIFTI PT
EXPORT NIFTI CT
708
EXPORT NIFTI PT
EXPORT NIFTI CT
709
EXPORT NIFTI PT
EXPORT NIFTI CT
710
EXPORT NIFTI PT
EXPORT NIFTI CT
711
EXPORT NIFTI PT
EXPORT NIFTI